<a href="https://colab.research.google.com/github/martinpovolny/colaboratory/blob/master/Cars_vs_trucks_w_o_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://ujc.hmpf.cz/download/cars-sorted-2019-06-01.zip  -O /tmp/cars-sorted.zip

--2019-06-01 10:22:18--  http://ujc.hmpf.cz/download/cars-sorted-2019-06-01.zip
Resolving ujc.hmpf.cz (ujc.hmpf.cz)... 147.231.205.34
Connecting to ujc.hmpf.cz (ujc.hmpf.cz)|147.231.205.34|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66101608 (63M) [application/zip]
Saving to: ‘/tmp/cars-sorted.zip’

/tmp/cars-sorted.zi 100%[===================>]  63.04M  11.2MB/s    in 6.6s    

2019-06-01 10:22:25 (9.51 MB/s) - ‘/tmp/cars-sorted.zip’ saved [66101608/66101608]



In [0]:
import os
import zipfile

local_zip = '/tmp/cars-sorted.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cars-sorted')
zip_ref.close()

In [0]:
train_dir = os.path.join('/tmp/cars-sorted/sorted')


In [0]:
image_size=(90, 160)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=128,
        class_mode='categorical')

# FIXME: loss_weights?


Found 4471 images belonging to 5 classes.


In [0]:
# import glob
# for file in glob.glob(train_dir + "/sorted/predek_osobni/*"):
#  print(file)


In [0]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential;
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation;

In [0]:
def model_4(image_size, num_classes):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(image_size[0], image_size[1], 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(rate = 0.5))
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))

    return model


In [0]:
model = model_4(image_size, 5)

In [0]:
from tensorflow.keras.optimizers import Adam, RMSprop

#model.compile(loss='binary_crossentropy',
#              optimizer=RMSprop(lr=0.001),
#              metrics=['acc'])

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(opt,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    loss_weights=None,
    sample_weight_mode=None,
    weighted_metrics=None,
    target_tensors=None)


In [36]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
35/35 [==============================] - 9s 244ms/step - loss: 1.2201 - acc: 0.5316
Epoch 2/15
35/35 [==============================] - 6s 177ms/step - loss: 0.8438 - acc: 0.6683
Epoch 3/15
35/35 [==============================] - 6s 178ms/step - loss: 0.6580 - acc: 0.7558
Epoch 4/15
35/35 [==============================] - 6s 177ms/step - loss: 0.5642 - acc: 0.7965
Epoch 5/15
35/35 [==============================] - 6s 177ms/step - loss: 0.4768 - acc: 0.8262
Epoch 6/15
35/35 [==============================] - 6s 179ms/step - loss: 0.4179 - acc: 0.8450
Epoch 7/15
35/35 [==============================] - 6s 181ms/step - loss: 0.3678 - acc: 0.8667
Epoch 8/15
35/35 [==============================] - 6s 181ms/step - loss: 0.3113 - acc: 0.8864
Epoch 9/15
35/35 [==============================] - 6s 178ms/step - loss: 0.2963 - acc: 0.8917
Epoch 10/15
35/35 [==============================] - 6s 178ms/step - loss: 0.2534 - acc: 0.9076
E

In [0]:
class_labels = (train_generator.class_indices)
class_labels = dict((v,k) for k,v in class_labels.items())

def from_categorical(x):
    return class_labels[np.argmax(x, axis=0)]

In [77]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=image_size)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(from_categorical(classes[0]))


Saving prijezd.down-1549009087-p.jpg to prijezd.down-1549009087-p.jpg
predek_osobni
